In [1]:
import os
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from scipy.io import wavfile as wav
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import progressbar
import time

In [2]:
data = pd.read_csv('metadata/Screamsound.csv')
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] > 0 ]
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')


In [13]:
from tqdm import tnrange, tqdm_notebook

D=[]
        
for row in tqdm_notebook(valid_data.itertuples()): 
    y1, sr1 = librosa.load("audio/" + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y1, sr=sr1)
    #print(ps)
    #print(ps.shape)
    
    if ps.shape != (128, 128): 
        print(row.path + "!=128,128")
        continue
        
    print(row.path)
    #print(row.classID)    
    D.append( (ps, row.classID) )

C:\Users\USER\AppData\Local\Temp\ipykernel_7708\3586499159.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(valid_data.itertuples()):


0it [00:00, ?it/s]

fold3/pk1e5-yogtt.wav
fold3/pk1e5-yogtt.wav
fold3/pk1e5-yogtt.wav
fold1/yisell_sound_2007_12_18_15_27_543156.wav
fold1/yisell_sound_2007_12_18_15_29_33722.wav
fold1/yisell_sound_2007_12_18_15_30_231364.wav
fold2/100652-3-0-0.wav
fold2/100652-3-0-1.wav
fold2/100652-3-0-2.wav
fold2/100652-3-0-3.wav
fold2/101415-3-0-2.wav
fold2/101415-3-0-3.wav
fold2/101415-3-0-8.wav
fold3/117048-3-0-17.wav
fold3/117048-3-0-23.wav
fold3/117048-3-0-25.wav
fold3/117048-3-0-35.wav
fold3/117072-3-0-14.wav
fold3/117072-3-0-19.wav
fold4/12917.wav
fold4/13238.wav
fold4/13238.wav
fold4/13548.wav
fold4/13548.wav
fold4/14510.wav
fold4/14510.wav
fold3/17615-3-0-0.wav
fold3/17615-3-0-3.wav
fold3/17615-3-0-4.wav
fold3/17615-3-0-6.wav
fold3/176783-3-0-11.wav
fold3/176783-3-0-3.wav
fold3/176783-3-0-5.wav
fold3/176783-3-1-0.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wav
fold4/3um4p-6li77.wa

In [4]:
#import hickle as hkl 
#data = D

## Dump data to file
#hkl.dump(data, 'new_data_file.hkl')

In [5]:
## Load data from file
#D = hkl.load('new_data_file.hkl')
#print(type(D))

In [6]:
#from keras.utils import np_utils
dataset = D
#random.shuffle(dataset)

#train = dataset[:15]
#test = dataset[15:]
print('dataset:', len(dataset))
#print('train:', len(train))
#print('test:', len(test))

dataset: 56


In [7]:
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split([i[0] for i in D],
                                                   [i[1] for i in D],
                                                   test_size=0.2,
                                                   random_state=0)

#X_train, y_train = zip(*train)
#X_test, y_test = zip(*test)

X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])
#print(y_train)
#print(y_test)
y_train = np.array(keras.utils.np_utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.np_utils.to_categorical(y_test, 10))
#print(y_train)
#print(y_test)

In [8]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

In [9]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=12,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('success')

Epoch 1/12
1/1 [==============================] - 1s 887ms/step - loss: 14.7876 - accuracy: 0.3409 - val_loss: 2.4651 - val_accuracy: 0.5000
Epoch 2/12
1/1 [==============================] - 0s 303ms/step - loss: 4.2161 - accuracy: 0.6136 - val_loss: 1.9880 - val_accuracy: 0.5000
Epoch 3/12
1/1 [==============================] - 0s 287ms/step - loss: 2.8340 - accuracy: 0.7045 - val_loss: 1.0620 - val_accuracy: 0.5833
Epoch 4/12
1/1 [==============================] - 0s 294ms/step - loss: 4.8847 - accuracy: 0.6591 - val_loss: 1.7458 - val_accuracy: 0.6667
Epoch 5/12
1/1 [==============================] - 0s 295ms/step - loss: 2.5634 - accuracy: 0.8409 - val_loss: 2.4235 - val_accuracy: 0.6667
Epoch 6/12
1/1 [==============================] - 0s 298ms/step - loss: 1.7042 - accuracy: 0.8182 - val_loss: 2.3489 - val_accuracy: 0.6667
Epoch 7/12
1/1 [==============================] - 0s 291ms/step - loss: 0.7484 - accuracy: 0.8636 - val_loss: 2.6717 - val_accuracy: 0.6667
Epoch 8/12
1/1 [===

In [10]:
predict = model.predict(X_test)
print(predict)

[[1.0000000e+00 1.2963960e-34 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [6.8113875e-01 1.9836886e-01 1.2413855e-02 1.0612893e-02 3.9984964e-02
  5.9492299e-03 2.6488166e-02 8.0905510e-03 1.3887821e-02 3.0649286e-03]
 [9.9994433e-01 5.5695313e-05 4.6525040e-13 1.5582415e-14 2.7271849e-09
  3.2565021e-16 1.4393077e-11 9.9788103e-12 3.5555697e-13 2.0674423e-15]
 [2.4739118e-01 4.9911055e-01 4.5709040e-02 2.9154446e-02 7.5415500e-02
  2.2013342e-02 3.1013235e-02 2.0029083e-02 1.7501716e-02 1.2661892e-02]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 7.7304387e-12 1.4358428e-29 3.0296349e-34 8.4856468e-19
  1.4715939e-35 5.2274859e-23 9.4308142e-26 1.3593545e-24 4.3780475e-33]
 [1.4148696e-01 1.3215142e-01 9.6706346e-02 8.4672861e-02 1.0131351e-01
  8.1219941e-02 9.3202569e-02 9.5896095e-02 8.9598805e-02 

In [11]:
predict = np.argmax(model.predict(X_test), axis=-1)

print(y_test)
print(predict)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[0 0 0 1 1 0 0 0 0 0 1 1]


In [12]:
model.save('ScreamDemo1.h5') 
print('Model exported and finished')

Model exported and finished
